In [1]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

from datetime import date
data_atual = date.today()

In [2]:
formato_numerico = "{0:10.2f}"
ativo = list(())
AtivoSymbolInfo = list(())
matriz = list(())
global matriz

In [3]:
def imprimirInfoAtivo(t, y, n):
    print(t, y, formato_numerico.format(n))

In [4]:
# ****************************************************
def RolarOpcoes(ativo):
    n = len(ativo)
    AtivoSymbolInfo.clear()
    rows,cols = (n,10)
    matriz = [[0 for i in range(cols)] for j in range(rows)]
    
    for i in range(n):
        AtivoSymbolInfo.append(mt5.symbol_info(ativo[i]))
        xx = AtivoSymbolInfo[i]
        matriz[i][0] = xx.name
        matriz[i][1] = xx.option_strike
        matriz[i][4] = xx.last
 
        # somente para linha 1, que é da opção vendida, que preciso rolar
        # pega o ASK... mas se não tiver ninguém oferecendo, pega o LAST
        if i == 1:
            matriz[1][3] = xx.ask
            if matriz[1][3] == 0:
                fff = input('Entre o valor do Ask ' + xx.name + ': ')
                matriz[i][3] = float(fff)
            gasto = matriz[1][3] * -1
#            if matriz[1][3] == 0:       gasto = matriz[1][4]
 
        # para as linhas 2,3,... ou seja, para as opções dos meses seguintes que poderei vender
        if i>1:
            matriz[i][2] = xx.bid
            if matriz[i][2] == 0:
                fff = input('Entre o valor do Bid ' + xx.name + ': ')
                matriz[i][2] = float(fff)
            matriz[i][5] = matriz[i][1] - matriz[1][1] # aumento em R$ strike nova - strike vendida
            matriz[i][6] = round((matriz[i][1]/matriz[1][1] - 1) * 100, 2)  # % strike nova - strike vendido
            matriz[i][7] = matriz[i][5] + gasto + matriz[i][2] # dif.strike - gasto
            matriz[i][8] = round(matriz[i][7] / matriz[1][1] * 100, 2)
            matriz[i][9] = matriz[i][2] - matriz[1][3] 

    df = pd.DataFrame(matriz, columns=['Ativo','Strike','bid','ask','Last','▲ R$','▲ % Strike','Somatório','% sobre '+matriz[1][0], 'Cash'])
    print(df)
    nome_arq = 'rolar_opcoes_' + ativo[0] + '_' + ativo[1] + '_' + ativo[2]+ '.xlsx'
    df.to_excel(nome_arq, sheet_name=str(data_atual))
    

In [5]:
print(type(data_atual))

<class 'datetime.date'>


In [6]:
### https://www.alura.com.br/artigos/lidando-com-datas-e-horarios-no-python?gclid=CjwKCAjwieuGBhAsEiwA1Ly_nWDbtS13ZSnMrNsQGuuQCRKckt2Jbyv8byErnftNStX5-SAf5fxwSxoCpF8QAvD_BwE

In [7]:
data_em_texto = '{}/{}/{}'.format(data_atual.day, data_atual.month,
data_atual.year)
data_em_texto

'30/7/2021'

In [8]:
ativo = ['IRBR3', 'IRBRT580', 'IRBRU580'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask  Last  ▲ R$  ▲ % Strike  Somatório  \
0     IRBR3     0.0  0.00  0.00  5.64   0.0         0.0       0.00   
1  IRBRT580     5.8  0.00  0.28  0.29   0.0         0.0       0.00   
2  IRBRU580     5.8  0.36  0.00  0.36   0.0         0.0       0.08   

   % sobre IRBRT580  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              1.38  0.08  


In [ ]:
RolarOpcoes(['BBSE3', 'BBSET230', 'BBSEU230'])

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     BBSE3     0.0  0.00  0.00  21.64   0.0        0.00       0.00   
1  BBSET230    23.0  0.00  1.35   1.34   0.0        0.00       0.00   
2  BBSEU220    22.0  0.69  0.00   0.69  -1.0       -4.35      -1.66   

   % sobre BBSET230  Cash  
0              0.00  0.00  
1              0.00  0.00  
2             -7.22 -0.66  
